In [3]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import NMF, TruncatedSVD
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt 

import pickle

In [4]:
file = open("full_df.pkl",'rb')
full_df = pickle.load(file)

In [22]:
full_df.to_csv('full_df.csv')

In [23]:
full_df

,cov_text_lower,datetime
0,july ¢ wealthy young students embr...,1964-07-01 00:00:00
1,may ° guided tour of the sac a compl...,1964-05-01 00:00:00
2,june idg ° mm cenatten dev ° oe ...,1964-06-01 00:00:00
3,july ¢ wealthy young students embr...,1964-07-01 00:00:00
4,august bn hes by shirley ann gr...,1964-08-01 00:00:00
5,pomel ari auch » copyright © pr...,1964-09-01 00:00:00
6,october oa parenthood wit...,1964-10-01 00:00:00
7,november ¢ please mr president ...,1964-11-01 00:00:00
8,december ¢ do religious schools teach ...,1964-12-01 00:00:00
9,vorking womens problems ty betty friedan w...,1965-01-01 00:00:00


In [6]:
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')
whitespace_tokenizer = RegexpTokenizer("\s+", gaps = True)

def tokenize(text):
    return [word for word in tokenizer.tokenize(text.lower())]

In [10]:
#establish and add on to stop words
st_words = stopwords.words('english')

my_words = (['proquest','cover','copyright','ee','eee','ae','©','«','»','e','oe','rights','new','york',
             'llc', 'all', 'rights','reserve','reserved','°','§','\"','\"','®',"\'"
             'cosmopolitan','president','subscription','service','vice','neutrogena',
            'barbara','richard','moines','william','box','offices','karen','nancy','liz',
            'hearst','copies','please','street','ellen','food','jane','susan','pat','cosmo',
            'fisher','street','avenue','helen','executive','assistants','one','editor','director',
            'issue','published','months','date','preceding','filing','get','editors',
            'sadowsky','john','robert','iowa','des','levine','ursula','nat','morgan',
            'cece','rubinstein','cee','helena','ccc',
            'abov', 'ani', 'assist', 'avenu', 'becaus', 'befor', 'copi', 'director',
             'issu', 'doe', 'dure', 'execut', 'file', 'levin', 'moin', 'month', 'nanci', 
             'offic', 'onc', 'onli', 'ourselv', 'pleas', 'preced', 'presid', 'publish', 
             'reserv', 'right', 'sadowski', 'servic', "should'v", 'subscript', 'themselv', 
             'veri', 'whi', "you'r", "you'v", 'yourselv','cosmopolitan', 'plea', 'prece',
            'catherin','dunn','joyc','oo','ooo','coc','occ','n','clapperton','j','g','jon',
            'anthoni', 'cc','rpm','terrac','q','iy','al','e','rae','rael','bee','rpc','arrose',
            'nee','est','alt','aan','pee','ove','aoe','vari','yee','een','ina','wee',
            'hee','eer','www','com','cant','eae','page','canada','june','december','january',
            'october','february','march','april','may','july','august','september','november',
            'tell','like'])#'man','woman','women','men','life','girl','girls'])
st_words.extend(my_words)

In [11]:
vectorizer = (CountVectorizer(stop_words = st_words,
                                 analyzer='word', token_pattern="\\b[a-z]{3,}\\b", 
                                  max_df=0.2, min_df=10))
X = vectorizer.fit_transform(full_df['cov_text_lower'])
words = np.array(vectorizer.get_feature_names())

model = NMF(n_components=6)
doc_topics = model.fit_transform(X)
C = model.components_

In [24]:
for topic_idx, topic in enumerate(C):
        print("Topic #%d:" % topic_idx)
        print(" ".join([words[i]
                    for i in topic.argsort()[:-15 - 1:-1]]))
        print()


Topic #0:
man life woman ways super way rich much affair know want quiz long relationship young

Topic #1:
best girl plus short stories girls story fiction life seller book diet two lover woman

Topic #2:
men women first excerpt big girl together lovers making stunning turn marriage sexual girls romance

Topic #3:
complete mystery murder husband see stories girls short money summer story keep suspense children special

Topic #4:
astrologer bedside money career guide cosmos health bonus star year plus beauty travel advice family

Topic #5:
make ways life sexy ever good bed hot know tricks things secret guys never feel



In [12]:
H = pd.DataFrame(doc_topics.round(5),
             index = [x for x in full_df['datetime']],
             columns = ["relaish","fiction","misc","mystery","resolutions","sex"])


In [14]:
H2 = H.sort_index()
H2 = H2.apply(lambda x: x/x.sum(), axis=1)
H2 = H2.dropna()
H2 = H2.reset_index()
H2 = H2.rename(index=str, columns={"index": "datetime"})

In [15]:
H3 = H2.groupby(H2.datetime.dt.year).mean()

In [8]:
full_df.to_pickle('full_df_save.pkl')


In [16]:
H2.to_pickle('H2_save.pkl')
H3.to_pickle('H3_save.pkl')

In [18]:
H3

,relaish,fiction,misc,mystery,resolutions,sex
datetime,,,,,,
1950,0.061862,0.166239,0.076283,0.437063,0.000000,0.258552
1951,0.001760,0.197492,0.069220,0.559424,0.040002,0.132102
1952,0.038768,0.223887,0.000515,0.458318,0.085379,0.193132
1953,0.165712,0.154527,0.106443,0.481628,0.021207,0.070483
1954,0.022035,0.233355,0.059916,0.524929,0.064570,0.095196
1955,0.155254,0.128177,0.025509,0.484176,0.095804,0.111079
1956,0.034535,0.283761,0.051367,0.402266,0.078737,0.149334
1957,0.019058,0.382549,0.003164,0.401296,0.173142,0.020790
1958,0.016025,0.263915,0.149758,0.492454,0.042205,0.035643


In [19]:
H2.groupby(H2.datetime.dt.year).count()

,datetime,relaish,fiction,misc,mystery,resolutions,sex
datetime,,,,,,,
1950,9,9,9,9,9,9,9
1951,12,12,12,12,12,12,12
1952,10,10,10,10,10,10,10
1953,10,10,10,10,10,10,10
1954,12,12,12,12,12,12,12
1955,11,11,11,11,11,11,11
1956,10,10,10,10,10,10,10
1957,11,11,11,11,11,11,11
1958,10,10,10,10,10,10,10
